In [ ]:
# IMporting Liberaries 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import wordcloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [ ]:
data = pd.read_csv("../input/training-data/toxic.csv")
data .head()

In [ ]:
doc=list(data['comment_text'].values[:200])

In [ ]:
doc

In [ ]:
  >>> import nltk
  >>> nltk.download('stopwords')

In [ ]:
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))

In [ ]:
doc

In [ ]:
stdoc=', '.join(doc)

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.summarization.textcleaner import split_sentences
all_stopwords_gensim = STOPWORDS
sw_list = {"not"}
all_stopwords_gensim = STOPWORDS.difference(sw_list)

doc_tokens = split_sentences(stdoc)
tokens_without_sw = [word for word in doc_tokens if not word in all_stopwords_gensim]

print(tokens_without_sw)

In [ ]:
import re

In [ ]:
doc_cleaned=[re.sub(r'[^\w]+',' ',i) for i in tokens_without_sw]

In [ ]:
tokens_without_sw[1],doc_cleaned[1]

In [ ]:
stdoc_cleaned=', '.join(doc_cleaned)
stdoc_cleaned

In [ ]:
data.head()

In [ ]:
#Creating a new Column that if a comment has any bad behavor or negative it carries valued 1 if not and its positive it will be valued by zero
data['Comment Behavor']=data['toxic']+data['severe_toxic']+data['obscene']+data['threat']+data['insult']+data['identity_hate']

In [ ]:
for i in data['Comment Behavor']:
    if i>=1:
        data['Comment Behavor'].replace(i,"toxic",inplace=True)
    elif i == 0:
        
        data['Comment Behavor'].replace(i,"courteous",inplace=True)
        

In [ ]:
# Visualizing 
from wordcloud import WordCloud, STOPWORDS 

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(stdoc_cleaned)

In [ ]:
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
data['comment_text'].head()

In [ ]:
# creating a bag of words (bow)

In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

def get_bag_of_words(doc_cleaned):
    vectorizer = CountVectorizer()
    print(vectorizer.fit_transform(doc_cleaned).toarray())
    print(vectorizer.vocabulary_) 

In [ ]:
 get_bag_of_words(doc_cleaned)

In [ ]:
# creating TFIDF Model 

In [ ]:
data["comment_text"].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer=TfidfVectorizer()

In [ ]:
doc_tfidf=vectorizer.fit_transform(doc_cleaned)

In [ ]:
print(doc_tfidf)

In [ ]:
len(doc_tfidf.A[0])

In [ ]:
len(doc_tfidf.A[1])

In [ ]:
# #applying LabelEncoding on [Comment Behavor] Column And convert it to list

from sklearn.preprocessing import LabelEncoder

comm_labels = LabelEncoder()
comm_enco=comm_labels.fit_transform(data['Comment Behavor'])


In [ ]:
comm_type=np.array(data['Comment Behavor'].values[:956])

In [ ]:
comm_type.shape

In [ ]:
doc_tfidf.shape

In [ ]:
# Handling Imbalanced Data

In [ ]:
 from collections import Counter
 from sklearn.datasets import make_classification
 X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                            n_redundant=0, n_repeated=0, n_classes=3,
                            n_clusters_per_class=1,
                            weights=[0.01, 0.05, 0.94],
                            class_sep=0.8, random_state=0)
 print(sorted(Counter(y).items()))

 from imblearn.under_sampling import ClusterCentroids
 cc = ClusterCentroids(random_state=0)
 X_resampled, y_resampled = cc.fit_resample(X, y)
 print(sorted(Counter(y_resampled).items()))


In [ ]:
 from imblearn.under_sampling import RandomUnderSampler
 rus = RandomUnderSampler(random_state=0)
 X_resampled, y_resampled = rus.fit_resample(X, y)
 print(sorted(Counter(y_resampled).items()))


In [ ]:
 import numpy as np
 print(np.vstack([tuple(row) for row in X_resampled]).shape)

 rus = RandomUnderSampler(random_state=0, replacement=True)
 X_resampled, y_resampled = rus.fit_resample(X, y)
print(np.vstack(np.unique([tuple(row) for row in X_resampled], axis=0)).shape)


In [ ]:
>>> X_hetero =(doc_tfidf)
>>> y_hetero =comm_type
>>> X_resampled, y_resampled = rus.fit_resample(X_hetero, y_hetero)
>>> print(X_resampled)

>>> print(y_resampled)


In [ ]:
 from imblearn.under_sampling import NearMiss
 nm1 = NearMiss(version=1)
 X_resampled_nm1, y_resampled = nm1.fit_resample(X, y)
 print(sorted(Counter(y_resampled).items()))

In [ ]:
# Splitting data as a preparation for Modeling 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train ,y_test = \
    train_test_split(doc_tfidf, comm_type, test_size=0.3, random_state=5)

In [ ]:
# Scaling 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.A)
X_test  = scaler.transform(X_test.A)

In [ ]:
toxic_detect_model = MultinomialNB().fit(X_train, y_train)
pred_test_MNB = toxic_detect_model.predict(X_test)

In [ ]:
acc_MNB = accuracy_score(y_test, pred_test_MNB)
print(acc_MNB)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(confusion_matrix(y_test, pred_test_MNB))

In [ ]:
#Modeling 

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

In [ ]:
acc_MNB = accuracy_score(y_test, pred_test_MNB)
print(acc_MNB)

In [ ]:
plot_confusion_matrix(confusion_matrix(y_test, pred_test_MNB))

In [ ]:

import pickle 


filename = 'finalized_model.sav'
pickle.dump(logreg, open(filename, 'wb'))


loaded_model = pickle.load(open(filename, 'rb'))


loaded_model.predict(X_test)

# # Save the trained model as a pickle string. 
# saved_model = pickle.dumps(SGDClassifier) 
  
# # Load the pickled model 
# SGDClassifier_from_pickle = pickle.loads(saved_model) 
  
# # Use the loaded pickled model to make predictions 
# SGDClassifier_from_pickle.predict(X_test) 